In [104]:
import findspark
findspark.init("/home/adam/Documents/spark/sp")
import pyspark 
findspark.find()

### Importation des librairies
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [107]:
##  Initialisation SparkSession
spark=SparkSession\
                  .builder.master("local[3]")\
                  .appName("Kmeans_Clustering")\
                  .getOrCreate()





In [106]:

import configparser
config = configparser.ConfigParser()
config.read('properties.conf')

['properties.conf']

In [95]:

path_to_input_data   = config['Bristol-City-bike']['Input-data']
path_to_output_data  = config['Bristol-City-bike']['Output-data']
num_partition_kmeans = config['Bristol-City-bike']['Kmeans-level'] 
num_partition_kmeans  = int(num_partition_kmeans )


In [96]:
bristol = spark.read.json(path_to_input_data)

In [97]:
bristol.show(2)

+--------------------+----------+----------+--------------------+------+
|             address|  latitude| longitude|                name|number|
+--------------------+----------+----------+--------------------+------+
|Lower River Tce /...|-27.482279|153.028723|122 - LOWER RIVER...|   122|
|Main St / Darragh St| -27.47059|153.036046|91 - MAIN ST / DA...|    91|
+--------------------+----------+----------+--------------------+------+
only showing top 2 rows



In [98]:
Kmeans_df=bristol.select("latitude","longitude")
Kmeans_df.show(2)

+----------+----------+
|  latitude| longitude|
+----------+----------+
|-27.482279|153.028723|
| -27.47059|153.036046|
+----------+----------+
only showing top 2 rows



In [99]:

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
features = ('longitude','latitude')
kmeans = KMeans().setK(num_partition_kmeans).setSeed(1)
assembler = VectorAssembler(inputCols=features,outputCol="features")
dataset=assembler.transform(Kmeans_df)
model = kmeans.fit(dataset)
fitted = model.transform(dataset)


In [100]:
fitted.columns

['latitude', 'longitude', 'features', 'prediction']

In [101]:
moy_DSL=fitted\
               .groupBy("prediction")\
               .mean("latitude","longitude")\
               .show()


+----------+-------------------+------------------+
|prediction|      avg(latitude)|    avg(longitude)|
+----------+-------------------+------------------+
|         1|-27.460240636363633|153.04186302272726|
|         2| -27.47255990624999|   153.02594553125|
|         0|-27.481218536585374|153.00572882926832|
+----------+-------------------+------------------+



In [102]:
fitted.createOrReplaceTempView("fittedSQL") # transformation du data frame en table !
spark.sql(""" select prediction, 
                 mean(latitude) as Moy_Latitude,
                 mean(longitude) as Moy_Longitude
           from fittedSQL
           group by prediction   """).show()

+----------+-------------------+------------------+
|prediction|       Moy_Latitude|     Moy_Longitude|
+----------+-------------------+------------------+
|         1|-27.460240636363633|153.04186302272726|
|         2| -27.47255990624999|   153.02594553125|
|         0|-27.481218536585374|153.00572882926832|
+----------+-------------------+------------------+



In [103]:
fitted = fitted.drop('features')
fitted = fitted.write.format("csv").mode("overwrite")\
    .save(path_to_output_data, header = 'true')

